Hurricane Analysis 

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
import itertools

from sqlalchemy import create_engine
import requests
from bs4 import BeautifulSoup

import calendar
from datetime import datetime, date, timedelta

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from textblob import TextBlob, Word
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

from keras.models import Sequential
from keras.layers import Dense

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


Data Set is webscraped Tornados in US, from 1950 - 2018

Dummies
regression 
Random Forest
confusion matrix 
Gridsearch
compile model 

In [12]:
#Import Data by connecting to storms Database With SQLalchemy
Tornado = pd.read_csv('Resources/CleanTornado2.csv')
Tornado.head()

,Unnamed: 0,SOURCE,TOR_F_SCALE,TOR_LENGTH,TOR_WIDTH,Duration,AverageDate,AverageTime,Deaths,Injuries,Mean_Lat,Mean_Lon,Azimuth,State,PropertyDamageCost,CropsDamageCost
0,0,Law_Enforcement,EF0,1.43,50.0,3,126,15.685,0,0,37.2945,-98.6804,1.11,KS,0,0
1,1,Broadcast_Media,EF0,1.38,20.0,5,183,17.535,0,0,26.7200,-80.2500,0.00,FL,0,0
2,2,Storm_Chaser,EF0,2.77,100.0,9,111,18.875,0,0,43.6830,-98.3963,81.93,SD,0,0
3,3,NWS_Storm_Survey,EF0,0.55,30.0,1,152,11.925,0,0,24.6658,-81.5240,-36.57,FL,20000,0
4,4,NWS_Storm_Survey,EF2,1.50,300.0,0,105,6.250,0,0,30.5300,-82.2500,64.36,FL,0,0


In [171]:
# # Drop rows that have nan
# Tornado.dropna()

In [13]:
Tornado = Tornado.reset_index()

In [14]:
y1 = Tornado.reset_index().TOR_F_SCALE
X1 = Tornado.reset_index().drop(['TOR_F_SCALE'], axis=1)

In [15]:
# Standardizing numerical features:
numfeat = ['TOR_LENGTH', 'TOR_WIDTH', 'Duration', 'AverageDate', 'AverageTime', 
           'Deaths', 'Injuries', 'Mean_Lat', 'Mean_Lon', 'Azimuth']
scaler = StandardScaler()
X1numfeat = pd.DataFrame(scaler.fit_transform(X1[numfeat]), columns=numfeat)
X1numfeat.head(2)

,TOR_LENGTH,TOR_WIDTH,Duration,AverageDate,AverageTime,Deaths,Injuries,Mean_Lat,Mean_Lon,Azimuth
0,-0.384696,-0.453753,-0.372725,-0.399389,0.034001,-0.046745,-0.057794,0.025597,-0.818166,-1.299679
1,-0.396503,-0.557985,-0.080881,0.326020,0.391832,-0.046745,-0.057794,-2.195896,1.453279,-1.324643


In [160]:
# x = Tornado.reset_index().drop(['TOR_F_SCALE'],axis=1)
# y = Tornado.reset_index().TOR_F_SCALE
# print(X.shape, y.shape)

(16353, 6) (16353,)


In [16]:
Tornado.State.value_counts().head(2)

TX    1551
KS    1227
Name: State, dtype: int64

In [17]:
Tornado.SOURCE.value_counts().head(2)

NWS_Storm_Survey    10349
Trained_Spotter      1167
Name: SOURCE, dtype: int64

In [18]:
Xdummies = pd.get_dummies(X1[['State','SOURCE']], prefix=['State','Source'],
                        columns=['State','SOURCE'])
Xdummies.drop(['State_TX','Source_NWS_Storm_Survey'], axis=1, inplace=True)
Xdummies.head(2)

,State_AL,State_AR,State_AZ,State_CA,State_CO,State_CT,State_DC,State_DE,State_FL,State_GA,...,Source_Official_NWS_Observations,Source_Other_Federal_Agency,Source_Park/Forest_Service,Source_Public,Source_Social_Media,Source_State_Official,Source_Storm_Chaser,Source_Trained_Spotter,Source_Unknown,Source_Utility_Company
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
X1 = pd.concat([X1numfeat,Xdummies], axis=1)
print(X1.shape, X1numfeat.shape, Xdummies.shape)
X1.head(2)

(16353, 86) (16353, 10) (16353, 76)


,TOR_LENGTH,TOR_WIDTH,Duration,AverageDate,AverageTime,Deaths,Injuries,Mean_Lat,Mean_Lon,Azimuth,...,Source_Official_NWS_Observations,Source_Other_Federal_Agency,Source_Park/Forest_Service,Source_Public,Source_Social_Media,Source_State_Official,Source_Storm_Chaser,Source_Trained_Spotter,Source_Unknown,Source_Utility_Company
0,-0.384696,-0.453753,-0.372725,-0.399389,0.034001,-0.046745,-0.057794,0.025597,-0.818166,-1.299679,...,0,0,0,0,0,0,0,0,0,0
1,-0.396503,-0.557985,-0.080881,0.326020,0.391832,-0.046745,-0.057794,-2.195896,1.453279,-1.324643,...,0,0,0,0,0,0,0,0,0,0


In [31]:
Xtrain1, Xtest1, ytrain1, ytest1 = train_test_split(X1, y1, test_size=0.3, 
                                                    stratify=y1, random_state=31)
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [39]:
ytest_valcounts = ytest1.value_counts()

In [40]:
Test1 = ytest1.value_counts()
accuracy_baseline = ytest_valcounts.get_values()[0]/float(ytest_valcounts.get_values().sum())
round(accuracy_baseline, 3)

0.527

In [42]:
LordOfTheRings1 = LogisticRegression()
accuracy_cv5_lr1 = np.mean(cross_val_score(LordOfTheRings1, Xtrain1, ytrain1, cv=5))
round(accuracy_cv5_lr1, 3)

0.669

In [43]:
#GRIDSEARCHING
target_names = ["negative", "positive"]
from sklearn.svm import SVC 
model = SVC(kernel='linear')

In [44]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10],
              'gamma': [0.0001, 0.001, 0.01]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [45]:
grid.fit(Xtrain1, ytrain1)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ...... C=1, gamma=0.0001, score=0.6733892090099528, total=   7.5s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.1s remaining:    0.0s


[CV] ...... C=1, gamma=0.0001, score=0.6771488469601677, total=   8.0s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   23.1s remaining:    0.0s


[CV] ...... C=1, gamma=0.0001, score=0.6923682140047207, total=   8.0s
[CV] C=1, gamma=0.001 ................................................
[CV] ....... C=1, gamma=0.001, score=0.6733892090099528, total=   7.3s
[CV] C=1, gamma=0.001 ................................................
[CV] ....... C=1, gamma=0.001, score=0.6771488469601677, total=   7.4s
[CV] C=1, gamma=0.001 ................................................
[CV] ....... C=1, gamma=0.001, score=0.6923682140047207, total=   7.8s
[CV] C=1, gamma=0.01 .................................................
[CV] ........ C=1, gamma=0.01, score=0.6733892090099528, total=   7.4s
[CV] C=1, gamma=0.01 .................................................
[CV] ........ C=1, gamma=0.01, score=0.6771488469601677, total=   7.6s
[CV] C=1, gamma=0.01 .................................................
[CV] ........ C=1, gamma=0.01, score=0.6923682140047207, total=   7.4s
[CV] C=5, gamma=0.0001 ...............................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  6.5min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [46]:
print(grid.best_params_)
# rf3.best_params_

{'C': 5, 'gamma': 0.0001}


In [47]:
print(grid.best_score_)

0.6812265222328995


In [48]:
predictions = grid.predict(Xtest1)

In [49]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(ytest1, predictions,
                            target_names=["1", "2","3","4","5","6","7"]))

              precision    recall  f1-score   support

           1       0.75      0.89      0.81      2585
           2       0.58      0.58      0.58      1642
           3       0.48      0.16      0.24       475
           4       0.66      0.17      0.27       134
           5       0.50      0.16      0.24        31
           6       0.25      0.25      0.25         4
           7       0.00      0.00      0.00        35

   micro avg       0.68      0.68      0.68      4906
   macro avg       0.46      0.32      0.34      4906
weighted avg       0.66      0.68      0.66      4906



In [50]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(Xtrain1, ytrain1)
rf.score(Xtest1, ytest1)

0.7105584997961679

In [51]:
feature_names = Tornado.columns

In [52]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.17388526817628988, 'Unnamed: 0'),
 (0.13006614635805303, 'index'),
 (0.09744011762494989, 'AverageTime'),
 (0.08701698221045646, 'TOR_LENGTH'),
 (0.0859873075899143, 'AverageDate'),
 (0.08133186248715414, 'Deaths'),
 (0.07904291920696076, 'SOURCE'),
 (0.07726119771948231, 'TOR_F_SCALE'),
 (0.03279634318104328, 'Duration'),
 (0.009385485273055248, 'TOR_WIDTH'),
 (0.004902805551909016, 'Injuries'),
 (0.004273516814419093, 'Mean_Lat'),
 (0.002274168440947871, 'State'),
 (0.000543691490392522, 'PropertyDamageCost'),
 (0.0005394288050391903, 'Azimuth'),
 (0.00025637012770212016, 'Mean_Lon'),
 (3.304487754141204e-05, 'CropsDamageCost')]

In [53]:
import scipy
print('scipy:%s' % scipy.__version__)

scipy:1.1.0


In [54]:
import numpy
import matplotlib
import pandas
import sklearn
import statsmodels

In [57]:
data = Tornado
y = data

In [8]:
#defining Validation Dataset
import warnings
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [5]:
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

Examples of parameter combinations for Seasonal ARIMA...
SARIMAX: (0, 0, 1) x (0, 0, 1, 12)
SARIMAX: (0, 0, 1) x (0, 1, 0, 12)
SARIMAX: (0, 1, 0) x (0, 1, 1, 12)
SARIMAX: (0, 1, 0) x (1, 0, 0, 12)


In [7]:
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue